In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer
import re
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
import keras
import cdsapi

2024-05-05 22:59:02.795717: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-05 22:59:02.896295: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-05 22:59:02.896411: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-05 22:59:02.910461: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-05 22:59:02.955839: I tensorflow/core/platform/cpu_feature_guar

In [3]:
# csv 파일을 dataframe으로 변환
df_outfit = pd.read_csv('../data/outfit(male)/outfit(male).csv')
df_weather = pd.read_csv('../data/2022-08-01_to_2024-04-30.csv', encoding='cp949')
# 필요한 columns만 추출
df_outfit = df_outfit[['userId', '상의', '아우터', '하의', '신발', '액세서리', '작성일']].copy()
df_temp = df_weather[['일시', '평균기온(°C)']].copy()

# '작성일'과 '일시' 열을 datetime 형식으로 변환
df_outfit['작성일'] = pd.to_datetime(df_outfit['작성일'], format='%Y년 %m월 %d일')
df_temp['일시'] = pd.to_datetime(df_temp['일시'])

# 두 dataframe을 날짜를 기준으로 병합
df_merged = pd.merge(df_outfit, df_temp, left_on='작성일', right_on='일시').drop('일시', axis=1)

df_merged

,userId,상의,아우터,하의,신발,액세서리,작성일,평균기온(°C)
0,1,"반팔 티, 셔츠/블라우스",재킷,반바지,구두/로퍼,NaN,2024-04-24,13.2
1,1,반팔 티,재킷,반바지,운동화,기타 모자,2024-04-19,17.6
2,1,반팔 티,재킷,반바지,구두/로퍼,장목양말,2024-04-15,16.0
3,1,반팔 티,NaN,나일론 팬츠,구두/로퍼,NaN,2024-04-09,15.3
4,1,반팔 티,집업,면바지,구두/로퍼,NaN,2024-04-05,14.0
...,...,...,...,...,...,...,...,...
1333,14,반팔 티,NaN,반바지,운동화,"기타 모자, 장목양말",2024-04-23,17.3
1334,14,반팔 티,NaN,카고바지,운동화,기타 모자,2024-04-24,13.2
1335,14,반팔 티,집업,나일론 팬츠,스니커즈/캔버스,기타 모자,2024-04-25,14.4
1336,14,"반팔 티, 셔츠/블라우스",NaN,반바지,구두/로퍼,장목양말,2024-04-26,17.8


In [4]:
# '상의', '아우터', '하의', '신발', '엑세서리' 열의 결측값을 '~ 없음'으로 대체
columns = ['상의', '아우터', '하의', '신발', '액세서리']
df_notnull = df_merged.copy()
for column in columns:
    df_notnull[column] = df_merged[column].fillna(column + ' 없음')

In [5]:
'''df_notnull[df_notnull['아우터'].str.contains('재킷 2')]'''

"df_notnull[df_notnull['아우터'].str.contains('재킷 2')]"

In [6]:
'''df_dup[df_dup['아우터'].str.contains('니트')]'''

"df_dup[df_dup['아우터'].str.contains('니트')]"

In [7]:
# 2가 붙은 단어를 두 번 반복하는 함수
def duplicate_word(text):
    words = text.split(', ')
    for i, word in enumerate(words):
        if '2' in word:
            words[i] = word.replace('2', '') + ', ' + word.replace('2', '')
    return ', '.join(words)

In [8]:
# 2가 붙은 단어를 두 번 반복한 dataframe df_dup 생성
df_dup = df_notnull.copy()
for column in columns:
    df_dup[columns] = df_notnull[columns].map(duplicate_word)

In [9]:
df_dup.columns

Index(['userId', '상의', '아우터', '하의', '신발', '액세서리', '작성일', '평균기온(°C)'], dtype='object')

In [10]:
# 옷의 조합 컬럼 생성 (상의, 아우터, 하의, 신발, 엑세서리의 각 값들을 하나의 문자열로 조합하여 하나의 컬럼으로 만듦)
df_combination = df_dup.copy()
df_combination['옷 조합'] = df_dup['상의'] + ', ' + df_dup['아우터'] + ', ' + df_dup['하의'] + ', ' + df_dup['신발'] + ', ' + df_dup['액세서리']
df_combination.drop(columns=['상의', '아우터', '하의', '신발', '액세서리'], inplace=True)

In [11]:
df_combination.columns

Index(['userId', '작성일', '평균기온(°C)', '옷 조합'], dtype='object')

In [12]:
'''df_combination[df_combination['옷 조합'].str.contains('니트 , 니트')]'''

"df_combination[df_combination['옷 조합'].str.contains('니트 , 니트')]"

In [13]:
'''# 옷의 조합 컬럼의 공백 제거
df_combination['옷 조합'] = df_combination['옷 조합'].str.replace(' ', '')'''

"# 옷의 조합 컬럼의 공백 제거\ndf_combination['옷 조합'] = df_combination['옷 조합'].str.replace(' ', '')"

In [14]:
# 쉼표를 기준으로 텍스트를 나누는 함수
def comma_tokenizer(s):
    return s.split(', ')

vectorizer = CountVectorizer(tokenizer=comma_tokenizer)

O = vectorizer.fit_transform(df_combination['옷 조합'])

# multi-hot encoding된 데이터를 numpy array로 변환
df_encoded = pd.DataFrame(O.toarray().tolist(), columns=vectorizer.get_feature_names_out())
npa = np.array(df_encoded)
npa.shape

/home/ksy/anaconda3/envs/ondoset/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(1338, 45)

In [15]:
# 값이 2 이상인 행의 인덱스
rows_with_value_2 = df_encoded[(df_encoded >= 2).any(axis=1)]
rows_with_value_2.index

Index([86, 317, 430, 435, 560, 593, 633, 640, 793, 1039], dtype='int64')

In [16]:
# 값이 2 이상인 열의 이름을 찾습니다.
columns_with_value_over_2 = df_encoded.columns[(df_encoded >= 2).any()]

# 특정 행에 대해 이를 기록합니다.
record = df_encoded.loc[rows_with_value_2.index, columns_with_value_over_2]
record

,니트,민소매 티,반팔 티,재킷
86,0,2,1,1
317,2,0,0,0
430,0,0,2,0
435,0,0,2,0
560,0,0,0,2
593,0,0,2,0
633,1,0,2,1
640,0,0,1,2
793,2,0,0,0
1039,0,0,2,0


In [17]:
# 단어장 확인
vectorizer.get_feature_names_out()

array(['가디건', '가죽 바지', '구두/로퍼', '기타 모자', '긴팔 티', '나일론 팬츠', '니트', '데님팬츠',
       '레더부츠', '레인부츠', '마스크', '맨투맨', '머플러', '면바지', '민소매 티', '바람막이', '반바지',
       '반팔', '반팔 니트', '반팔 셔츠/블라우스', '반팔 티', '비니', '샌들/슬리퍼', '셔츠/블라우스',
       '스니커즈/캔버스', '스카프', '슬랙스', '아우터 없음', '액세서리 없음', '양말', '운동화', '장목양말',
       '재킷', '점퍼', '조끼', '집업', '카고바지', '코트', '털 모자', '트레이닝/조거 팬츠', '패딩',
       '패딩슈즈', '패딩조끼', '하의 없음', '후드티'], dtype=object)

In [18]:
# numpy array를 list로 변환 후 clothes_combination 컬럼에 대입
df_combination['옷 조합'] = npa.tolist()
df_combination['옷 조합']

0       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3       [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...
                              ...                        
1333    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1334    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1335    [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1336    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1337    [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...
Name: 옷 조합, Length: 1338, dtype: object

In [19]:
# multi-hot encoding된 데이터를 다시 텍스트로 변환
df_combination['옷 조합'] = vectorizer.inverse_transform(npa)
df_combination['옷 조합'] 

0        [구두/로퍼, 반바지, 반팔 티, 셔츠/블라우스, 액세서리 없음, 재킷]
1                     [기타 모자, 반바지, 반팔 티, 운동화, 재킷]
2                    [구두/로퍼, 반바지, 반팔 티, 장목양말, 재킷]
3          [구두/로퍼, 나일론 팬츠, 반팔 티, 아우터 없음, 액세서리 없음]
4                 [구두/로퍼, 면바지, 반팔 티, 액세서리 없음, 집업]
                          ...                    
1333        [기타 모자, 반바지, 반팔 티, 아우터 없음, 운동화, 장목양말]
1334             [기타 모자, 반팔 티, 아우터 없음, 운동화, 카고바지]
1335          [기타 모자, 나일론 팬츠, 반팔 티, 스니커즈/캔버스, 집업]
1336    [구두/로퍼, 반바지, 반팔 티, 셔츠/블라우스, 아우터 없음, 장목양말]
1337             [기타 모자, 데님팬츠, 반팔 티, 아우터 없음, 운동화]
Name: 옷 조합, Length: 1338, dtype: object

In [20]:
# 하나의 문자열로 변환
df_combtest = df_combination.copy()
df_combtest['옷 조합'] = df_combination['옷 조합'].apply(lambda x: ', '.join(map(str, x)))

In [21]:
# multi-hot encoding의 값이 2 이상인 경우, 해당 단어를 두 번 반복
for i in record.index:
    old_value = df_combtest.loc[i, '옷 조합']
    for col in record.columns:
        if record.loc[i, col] >= 2:
            old_value = old_value.replace(col, col + ', ' + col)
    df_combtest.loc[i, '옷 조합'] = old_value

In [22]:
df_combtest.loc[rows_with_value_2.index, '옷 조합']

86      기타 모자, 데님팬츠, 민소매 티, 민소매 티, 반팔 티, 셔츠/블라우스, 스니커즈...
317               니트, 니트, 데님팬츠, 스니커즈/캔버스, 아우터 없음, 액세서리 없음
430           데님팬츠, 반팔 티, 반팔 티, 스니커즈/캔버스, 아우터 없음, 액세서리 없음
435              면바지, 반팔 티, 반팔 티, 샌들/슬리퍼, 아우터 없음, 액세서리 없음
560               구두/로퍼, 기타 모자, 데님팬츠, 셔츠/블라우스, 재킷, 재킷, 조끼
593                    반바지, 반팔 티, 반팔 티, 아우터 없음, 운동화, 장목양말
633               구두/로퍼, 니트, 면바지, 반팔 티, 반팔 티, 액세서리 없음, 재킷
640                     구두/로퍼, 면바지, 반팔 티, 액세서리 없음, 재킷, 재킷
793                      니트, 니트, 비니, 스니커즈/캔버스, 카고바지, 패딩조끼
1039              반바지, 반팔 티, 반팔 티, 스니커즈/캔버스, 아우터 없음, 장목양말
Name: 옷 조합, dtype: object

In [23]:
# 중복된 행 찾기
duplicated_rows = df_combtest[df_combtest.duplicated(['옷 조합'], keep=False)]
duplicated_rows.sort_values(by='옷 조합')

,userId,작성일,평균기온(°C),옷 조합
1008,12,2024-01-06,0.7,"가디건, 구두/로퍼, 긴팔 티, 면바지, 액세서리 없음"
1058,13,2022-11-21,11.8,"가디건, 구두/로퍼, 긴팔 티, 면바지, 액세서리 없음"
188,2,2023-02-15,3.2,"가디건, 구두/로퍼, 긴팔 티, 면바지, 액세서리 없음"
1014,12,2024-02-16,2.3,"가디건, 구두/로퍼, 긴팔 티, 면바지, 액세서리 없음, 재킷"
1009,12,2024-01-20,3.4,"가디건, 구두/로퍼, 긴팔 티, 면바지, 액세서리 없음, 재킷"
...,...,...,...,...
146,2,2023-07-28,29.5,"반팔 티, 아우터 없음, 액세서리 없음, 운동화, 트레이닝/조거 팬츠"
684,8,2023-08-29,23.9,"반팔 티, 아우터 없음, 액세서리 없음, 운동화, 트레이닝/조거 팬츠"
878,11,2023-09-20,20.6,"반팔 티, 아우터 없음, 액세서리 없음, 운동화, 트레이닝/조거 팬츠"
751,9,2023-12-20,-7.4,"아우터 없음, 액세서리 없음, 운동화, 카고바지, 후드티"


In [24]:
# pivot_table을 이용한 user-item matrix 생성
UI_matrix = df_combtest.pivot_table(values='평균기온(°C)', index='userId', columns='옷 조합', fill_value=0)
UI_matrix.columns

Index(['가디건, 구두/로퍼, 기타 모자, 긴팔 티, 장목양말, 트레이닝/조거 팬츠',
       '가디건, 구두/로퍼, 기타 모자, 긴팔 티, 코트, 트레이닝/조거 팬츠',
       '가디건, 구두/로퍼, 기타 모자, 데님팬츠, 반팔 티', '가디건, 구두/로퍼, 긴팔 티, 데님팬츠, 액세서리 없음',
       '가디건, 구두/로퍼, 긴팔 티, 데님팬츠, 액세서리 없음, 코트', '가디건, 구두/로퍼, 긴팔 티, 면바지, 액세서리 없음',
       '가디건, 구두/로퍼, 긴팔 티, 면바지, 액세서리 없음, 재킷',
       '가디건, 구두/로퍼, 니트, 데님팬츠, 머플러, 반팔 티, 코트', '가디건, 구두/로퍼, 니트, 데님팬츠, 액세서리 없음',
       '가디건, 구두/로퍼, 니트, 면바지, 비니',
       ...
       '비니, 셔츠/블라우스, 아우터 없음, 운동화, 트레이닝/조거 팬츠',
       '비니, 셔츠/블라우스, 운동화, 집업, 트레이닝/조거 팬츠',
       '셔츠/블라우스, 스니커즈/캔버스, 슬랙스, 액세서리 없음, 재킷',
       '셔츠/블라우스, 스니커즈/캔버스, 아우터 없음, 액세서리 없음, 트레이닝/조거 팬츠',
       '셔츠/블라우스, 슬랙스, 액세서리 없음, 운동화, 재킷',
       '스니커즈/캔버스, 아우터 없음, 액세서리 없음, 트레이닝/조거 팬츠, 후드티',
       '스니커즈/캔버스, 액세서리 없음, 재킷, 카고바지, 후드티',
       '스니커즈/캔버스, 액세서리 없음, 재킷, 트레이닝/조거 팬츠, 후드티',
       '스니커즈/캔버스, 액세서리 없음, 점퍼, 트레이닝/조거 팬츠, 후드티',
       '아우터 없음, 액세서리 없음, 운동화, 카고바지, 후드티'],
      dtype='object', name='옷 조합', length=696)

In [25]:
# pivot_table을 이용한 user_
UI_count = df_combtest.pivot_table( index='userId', columns='옷 조합', aggfunc='size', fill_value=0) 

In [26]:
# 평균기온을 5도 간격으로 하여 카테고리화하고 이를 '온도' 컬럼에 저장
df_temp_category = df_combtest.copy()
df_temp_category['온도'] = pd.cut(df_temp_category['평균기온(°C)'], bins=range(-10, 40, 5), right=False)
df_temp_category.to_csv('../data/outfit(male)/temp_category.csv', encoding='utf-8-sig')

In [27]:
# user-item
UI_matrix.to_csv('../data/outfit(male)/user_item_matrix.csv', encoding='utf-8-sig')

In [28]:
# user-item matrix에 기록된 값이 존재하는 경우 1, 아닌 경우 0으로 변환하여 R_df에 기록
R_df = UI_matrix.map(lambda x: 1 if x != 0 else 0)
R_np = np.array(R_df)
R_np.sum(axis=0)

array([1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 4, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1,
       1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 9, 1, 1,
       1, 3, 1, 1, 2, 1, 1, 1, 1, 3, 1, 1, 1, 2, 1, 1, 1, 2, 4, 3, 2, 3,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 2, 1, 1, 1, 1, 2, 2, 1,
       1, 1, 3, 5, 5, 1, 6, 1, 1, 1, 2, 1, 1, 2, 3, 1, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 1, 1, 3, 2, 2, 2, 1, 1, 5, 3, 1, 1, 2, 2, 1, 1, 2, 1, 2,
       4, 1, 2, 1, 3, 1, 1, 2, 1, 3, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 2,
       1, 2, 1, 1, 1, 3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 2, 1, 1, 1,
       1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 2, 1, 6, 3, 4,

In [29]:
# 각 열의 합이 2 이상(여러 유저가 해당 옷 조합을 선택한 경우)인 열을 찾음
columns_with_sum_over_2 = R_df.columns[R_df.sum() >= 2]

In [30]:
'''UI_matrix[columns_with_sum_over_2]'''

'UI_matrix[columns_with_sum_over_2]'

In [31]:
'''columns_with_sum_over_2'''

'columns_with_sum_over_2'

In [32]:
'''# 해당 조합의 인덱스
column_index = []
for i in columns_with_sum_over_2:
    column_index.append(R_df.columns.get_loc(i))
    column_index
column_index'''

'# 해당 조합의 인덱스\ncolumn_index = []\nfor i in columns_with_sum_over_2:\n    column_index.append(R_df.columns.get_loc(i))\n    column_index\ncolumn_index'

In [33]:
'''for row in R_np:
    print(row)'''

'for row in R_np:\n    print(row)'

In [34]:
# CF를 위한 초기값 설정
Y = np.array(UI_matrix)
Y = Y.T
count = np.array(UI_count)
count = count.T
print(Y.shape)
R = Y != 0 
n_u = Y.shape[1]
n_o = Y.shape[0]

(696, 14)


In [46]:
# 기록이 존재하는 값의 평균을 구함
o_sum = Y.sum(axis=1)
o_count = R.sum(axis=1)
o_mean = o_sum / o_count
o_mean.shape
o_mean = o_mean.reshape(-1, 1)
o_mean.shape

(696, 1)

In [36]:
'''Y[column_index]'''

'Y[column_index]'

In [39]:
Y_stand = Y - (o_mean * R)
'''Y_stand[column_index]'''

'Y_stand[column_index]'

In [40]:
def cofi_cost_func_v(O, U, b, Y, R, lambda_):
    j = (tf.linalg.matmul(O, tf.transpose(U)) + b - Y )*R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(O**2) + tf.reduce_sum(U**2))
    return J

In [41]:
# user, outfit의 수
n_o, n_u = Y.shape
# latent factor의 수
num_features = 30

# (U,O)를 초기화하고 tf.Variable로 등록하여 추적
tf.random.set_seed(1234) # for consistent results
U = tf.Variable(tf.random.normal((n_u,  num_features),dtype=tf.float64),  name='U')
O = tf.Variable(tf.random.normal((n_o, num_features),dtype=tf.float64),  name='O')
b = tf.Variable(tf.random.normal((1,          n_u),   dtype=tf.float64),  name='b')

# optimizer 초기화
optimizer = keras.optimizers.Adam(learning_rate=1e-1)

2024-05-05 20:08:03.173815: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-05 20:08:03.357794: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-05 20:08:03.357848: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-05 20:08:03.366902: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-05 20:08:03.366963: I external/local_xla/xla/stream_executor

In [42]:
J = cofi_cost_func_v(O, U, b, Y_stand, R, 1.5)

2024-05-05 20:08:04.337323: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


In [43]:
print(f"Cost (with regularization): {J:0.2f}")

Cost (with regularization): 36422.74


In [44]:
iterations = 200
lambda_ = 1
for iter in range(iterations):
    # TensorFlow의 GradientTape 사용
    # 연산을 기록하여 cost에 대한 gradient를 자동으로 계산
    with tf.GradientTape() as tape:

        # cost 계산 (forward pass included in cost)
        cost_value = cofi_cost_func_v(O, U, b, Y_stand, R, lambda_)

    # GradientTape를 통해 자동 미분
    # loss에 대한 trainable parameter의 gradient를 계산
    grads = tape.gradient( cost_value, [O,U,b] )

    # optimizer를 사용하여 trainable parameter를 업데이트
    optimizer.apply_gradients( zip(grads, [O,U,b]) )

    # Log periodically.
    if iter % 20 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:0.1f}")
        print(f'validation loss at iteration {iter}: {cofi_cost_func_v(O, U, b, Y_stand, R, lambda_):0.1f}')

2024-05-05 20:08:04.813010: I external/local_xla/xla/service/service.cc:168] XLA service 0xb7b2820 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-05 20:08:04.813053: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-05-05 20:08:04.841196: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-05 20:08:04.887052: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
I0000 00:00:1714907284.993317    1006 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Training loss at iteration 0: 31076.2
validation loss at iteration 0: 20470.5
Training loss at iteration 20: 2723.0
validation loss at iteration 20: 2540.0
Training loss at iteration 40: 834.2
validation loss at iteration 40: 795.6
Training loss at iteration 60: 421.2
validation loss at iteration 60: 412.9
Training loss at iteration 80: 331.0
validation loss at iteration 80: 329.1
Training loss at iteration 100: 308.7
validation loss at iteration 100: 308.2
Training loss at iteration 120: 301.7
validation loss at iteration 120: 301.5
Training loss at iteration 140: 298.8
validation loss at iteration 140: 298.7
Training loss at iteration 160: 297.3
validation loss at iteration 160: 297.3
Training loss at iteration 180: 296.4
validation loss at iteration 180: 296.4


In [45]:
# 예측을 수행하기 위해 모든 user-item에 대한 예측값을 계산
p = np.matmul(O.numpy(), np.transpose(U.numpy())) + b.numpy()
# 실제 온도
temp = 11
# 평균을 적용하고 temp를 빼서 값이 작을수록 실제 온도에 가깝도록 함. 이 때 각 user-item의 사용 횟수를 가중하여 많이 사용한 item이 추천되도록 함
pm = np.power(p + o_mean - temp, 2)  -count * 8

# 각 user마다 반복
for i in range(n_u) :
    my_predictions = pm[:,i]

    # sort predictions
    ix = tf.argsort(my_predictions, direction='ASCENDING')

    df_predict = UI_matrix[UI_matrix.columns[ix[0:10]]]
    df_predict.to_csv(f'../data/predictions/male/user_{i+1}_predictions.csv')

In [46]:
df_predict

옷 조합,"기타 모자, 데님팬츠, 바람막이, 반팔 티, 운동화","기타 모자, 긴팔 티, 데님팬츠, 레더부츠, 코트","기타 모자, 바람막이, 반바지, 반팔 티, 운동화, 장목양말","기타 모자, 데님팬츠, 반팔 티, 운동화, 점퍼","구두/로퍼, 기타 모자, 니트, 데님팬츠, 반팔 티, 아우터 없음","기타 모자, 긴팔 티, 데님팬츠, 레더부츠, 재킷","기타 모자, 반팔 티, 아우터 없음, 운동화, 카고바지","기타 모자, 반팔 티, 아우터 없음, 운동화, 카고바지, 후드티","구두/로퍼, 기타 모자, 데님팬츠, 반팔 티, 셔츠/블라우스, 아우터 없음","구두/로퍼, 기타 모자, 긴팔 티, 데님팬츠, 재킷"
userId,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,5.000000,0.0,0.0,0.00,7.4
2,0.0,0.0,0.0,0.0,0.0,0.000000,24.5,0.0,0.00,0.0
3,0.0,0.0,25.4,0.0,0.0,0.000000,0.0,0.0,0.00,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0
5,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0
6,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0
7,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0
8,0.0,0.0,0.0,0.0,0.0,0.000000,27.7,0.0,0.00,0.0
9,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,-2.2,0.00,0.0


In [47]:
UI_count['기타 모자, 긴팔 티, 데님팬츠, 레더부츠, 코트']

userId
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    4
Name: 기타 모자, 긴팔 티, 데님팬츠, 레더부츠, 코트, dtype: int64

In [48]:
df_combtest[df_combtest['옷 조합'].str.contains('기타 모자, 긴팔 티, 데님팬츠, 레더부츠, 코트')]

,userId,작성일,평균기온(°C),옷 조합
1226,14,2023-11-05,17.3,"기타 모자, 긴팔 티, 데님팬츠, 레더부츠, 코트"
1234,14,2023-11-22,9.5,"기타 모자, 긴팔 티, 데님팬츠, 레더부츠, 코트"
1239,14,2023-11-29,-1.4,"기타 모자, 긴팔 티, 데님팬츠, 레더부츠, 코트"
1255,14,2024-01-05,5.0,"기타 모자, 긴팔 티, 데님팬츠, 레더부츠, 코트"
